In [2]:
import pandas as pd
from postal.expand import expand_address

# **DPE ADEME**

### LOGEMENTS EXISTANTS ###

In [3]:
df = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-existants.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

df['Date_réception_DPE'].count()

4537525

On filtre pour n'avoir que les données du département 44 en 2022.

In [4]:
df['Date_réception_DPE'] = pd.to_datetime(df['Date_réception_DPE'], errors='coerce')
dfv1 = df[df['Date_réception_DPE'].dt.year == 2022].copy()
dfv2 = dfv1[dfv1['N°_département_(BAN)']=='44'].copy()

In [5]:
dfv2['Date_réception_DPE'].count()

64490

La fonction normalize renvoie des listes de différentes versions d'adresses possibles (ex Chateau-Thabut ; Chateau Thabut ; ChateauThabut)

Avec cette version de la fonction on ne garde que la première composante de la liste. 

In [6]:
def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        normalized = expand_address(address)  
        return normalized[0] if normalized else None  # Ne garde que la première version
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

In [7]:
dfv2['Adresse_Normalisee'] = dfv2['Adresse_(BAN)'].apply(normalize_address)

### LOGEMENTS NEUFS ###

In [8]:
df2 = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/DPE/DPE_ADEME/dpe-v2-logements-neufs.csv",
    sep=",",  # Séparateur CSV
    encoding="utf-8",
    low_memory=False)

df2['Date_réception_DPE'].count()

537952

In [9]:
df2['Date_réception_DPE'] = pd.to_datetime(df2['Date_réception_DPE'], errors='coerce')
df2v1 = df2[df2['Date_réception_DPE'].dt.year == 2022].copy()
df2v2 = df2v1[df2v1['N°_département_(BAN)']=='44'].copy()

In [10]:
df2v2['Date_réception_DPE'].count()

11624

In [11]:
df2v2['Adresse_Normalisee'] = df2v2['Adresse_(BAN)'].apply(normalize_address)

### VALEURS FONCIERES

In [12]:

vf = pd.read_csv(
    "/Users/noelinecasteil/Documents/statapp/ValeursFoncieres/valeursfoncieres-2022.txt",
    sep="|",  
    encoding="utf-8",
    low_memory=False
)

In [13]:
print(vf[['No voie', 'Type de voie', 'Voie', 'Code postal', 'Commune']].dtypes)


No voie         float64
Type de voie     object
Voie             object
Code postal     float64
Commune          object
dtype: object


In [14]:
# Convertir explicitement toutes les colonnes en chaînes
vf['Adresse'] = vf['No voie'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Type de voie'].fillna('').astype(str) + " " + \
                vf['Voie'].fillna('').astype(str) + ", " + \
                vf['Code postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['Commune'].fillna('').astype(str)

vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides


In [15]:
vf44 = vf[vf['Code postal'].notna()]
vf44 = vf44[vf44['Code postal'].astype(str).str.startswith('44')].copy()

In [16]:
vf44['Adresse'] = vf44['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)

In [17]:
vf44['Adresse_Normalisee'] = vf44['Adresse'].apply(normalize_address)

In [18]:
print(len(vf44['Adresse_Normalisee']))

99874


## **MATCHING AVEC PARSING**

In [19]:
def unique(df):
    adresse_counts = df['Adresse_Normalisee'].value_counts()
    nb_adresses_uniques = (adresse_counts == 1).sum()
    print("Nombre d'adresses uniques dans le fichier :")
    return nb_adresses_uniques

In [20]:
unique(dfv2)

Nombre d'adresses uniques dans le fichier :


22972

In [21]:
unique(vf44)

Nombre d'adresses uniques dans le fichier :


16910

In [22]:
print(f"Nombre d'adresses dupliquées dans dfv2 (logements existants) : {dfv2 ['Adresse_Normalisee'].value_counts().ge(2).sum()}")
print(f"Nombre d'adresses dupliquées dans df2v2 (logements neufs) : {df2v2 ['Adresse_Normalisee'].value_counts().ge(2).sum()}")
print(f"Nombre d'adresses dupliquées dans vf44 (valeurs foncières) : {vf44 ['Adresse_Normalisee'].value_counts().ge(2).sum()}")

Nombre d'adresses dupliquées dans dfv2 (logements existants) : 7513
Nombre d'adresses dupliquées dans df2v2 (logements neufs) : 707
Nombre d'adresses dupliquées dans vf44 (valeurs foncières) : 20674


TRAVAIL SUR LES DOUBLONS

DOUBLONS DANS DOC LOGEMENTS EXISTANTS

In [31]:
def calculate_duplicate_percentage(df, address_column):
    """
    On calcule le pourcentage d'adresses en doublon + on en fait une liste
    """
    total_count = len(df)
    duplicate_counts = df[address_column].value_counts()
    duplicate_addresses = duplicate_counts[duplicate_counts > 1].index.tolist()
    duplicate_count = len(duplicate_addresses)
    duplicate_percentage = (duplicate_count / total_count) * 100 if total_count > 0 else 0
    
    return duplicate_percentage, duplicate_addresses

# On applique la fonction aux logements existants
duplicate_percentage, duplicate_addresses = calculate_duplicate_percentage(dfv2, 'Adresse_Normalisee')

print(f"Pourcentage d'adresses en doublon : {duplicate_percentage:.2f}%")

liste = duplicate_addresses
print("Liste des adresses en doublon :")
print(liste[:10])

Pourcentage d'adresses en doublon : 11.65%
Liste des adresses en doublon :
['1 rue de cahors 44800 saint-herblain', '5 avenue robert chasteland 44700 orvault', '32 route de la joneliere 44300 nantes', '24 rue blaise pascal 44300 nantes', 'route de saint joseph 44300 nantes', '4 avenue des jades 44300 nantes', 'rue de la coran 44400 reze', '8 place francois ii 44200 nantes', '129 rue de la mirette 44400 reze', '5 rue de biarritz 44200 nantes']


DOUBLONS DANS VF44

In [32]:
def calculate_duplicate_percentage(df, address_column):
    """
    On calcule le pourcentage d'adresses en doublon + on en fait une liste
    """
    total_count = len(df)
    duplicate_counts = df[address_column].value_counts()
    duplicate_addresses2 = duplicate_counts[duplicate_counts > 1].index.tolist()
    duplicate_count = len(duplicate_addresses2)
    duplicate_percentage = (duplicate_count / total_count) * 100 if total_count > 0 else 0
    
    return duplicate_percentage, duplicate_addresses

# On applique la fonction aux logements existants
duplicate_percentage, duplicate_addresses2 = calculate_duplicate_percentage(vf44, 'Adresse_Normalisee')

print(f"Pourcentage d'adresses en doublon : {duplicate_percentage:.2f}%")

liste = duplicate_addresses2
print("Liste des adresses en doublon :")
print(liste[:10])

Pourcentage d'adresses en doublon : 20.70%
Liste des adresses en doublon :
['1 rue de cahors 44800 saint-herblain', '5 avenue robert chasteland 44700 orvault', '32 route de la joneliere 44300 nantes', '24 rue blaise pascal 44300 nantes', 'route de saint joseph 44300 nantes', '4 avenue des jades 44300 nantes', 'rue de la coran 44400 reze', '8 place francois ii 44200 nantes', '129 rue de la mirette 44400 reze', '5 rue de biarritz 44200 nantes']


CREATION DUNE LISTE AVEC LES ADRESSES UNIQUES ET EN DOUBLON POUR COMPTER NOMBRE DE MATCHING A EFFECTUER

In [33]:
adresse_counts = dfv2['Adresse_Normalisee'].value_counts()

# Séparer les adresses uniques et celles en doublon (qu'on ne garde qu'une seule fois)
adresses_uniques = adresse_counts[adresse_counts == 1].index.tolist()
adresses_doublons = adresse_counts[adresse_counts > 1].index.tolist()

# Fusionner les deux listes
adresses_finales = adresses_uniques + adresses_doublons

# Affichage du résultat
print("Liste des adresses uniques + adresses en doublon (mais une seule fois) :")
print(adresses_finales[:10])
print(len(adresses_finales))

Liste des adresses uniques + adresses en doublon (mais une seule fois) :
['7 rue de courtigon 44210 pornic', '46 rue de la bourdonnais 44100 nantes', '35 rue de la chicotiere 44800 saint-herblain', '23 avenue centre henri de cosse brissac 44540 vallons-de-lerdre', '51 rue du grand puits 44450 divatte-sur-loire', '10 rue georges clemenceau 44340 bouguenais', '48ter rue des primeveres 44220 coueron', '29 avenue des noes 44380 pornichet', '18 square des tilleuls 44522 mesanger', '23 rue herve bazin 44130 blain']
30485


In [27]:
adresse_counts = vf44['Adresse_Normalisee'].dropna().value_counts()


adresses_uniques = list(adresse_counts[adresse_counts == 1].index)
adresses_doublons = list(adresse_counts[adresse_counts > 1].index)

adresses_finales2 = adresses_uniques + adresses_doublons

print(f"Taille finale de la liste : {len(adresses_finales2)}")
print("Liste des adresses uniques + adresses en doublon (mais une seule fois) :")
print(adresses_finales2[:10])


Taille finale de la liste : 37584
Liste des adresses uniques + adresses en doublon (mais une seule fois) :
['l ouche du puits fresnay 44580 villeneuve-en-retz', 'le landier du ruaud 44410 asserac', '17 rue du puy civaux 44580 villeneuve-en-retz', 'les malabris 44130 notre dame des landes', '5362 le perron 44160 besne', '83 l annerie 44190 getigne', 'les landes de la piolais 44160 crossac', '3 rue du taillis 44140 montbert', '2 impasse des papillons 44450 saint-julien-de-concelles', '1 rue des chaloires 44760 la bernerie enceinte retz']


In [28]:
# Conversion des listes en ensemble
set_adresses_finales = set(adresses_finales)
set_adresses_finales2 = set(adresses_finales2)

# Adresses en commun
adresses_communes = set_adresses_finales.intersection(set_adresses_finales2)

print(f"Nombre d'adresses en commun : {len(adresses_communes)}")

# Exemple d'adresses en commun
print("Exemples d'adresses en commun :", list(adresses_communes)[:10])


Nombre d'adresses en commun : 7120
Exemples d'adresses en commun : ['134 route des frechets 44600 saint-nazaire', '9 rue scribe 44000 nantes', '6 boulevard francois blancho 44200 nantes', '11 rue albert vincon 44570 trignac', '32 rue eugene guillevic 44150 ancenis-saint-gereon', 'la champeliere 44540 vallons-de-lerdre', '35 rue saint benoit 44290 masserac', '31 rue des grands patis 44300 nantes', '32 route de pont caffino 44120 vertou', '8 rue du 8 mai 44130 blain']


MEILLEURE VISION DES DOUBLONS EN CSV

Attention à ne pas push sur git le fichier csv généré (il est trop lourd)

In [29]:
def export_duplicate_addresses(dfv2, vf44, output_file):
    # Identifier les adresses en doublon dans vf44
    vf44_dupes = vf44[vf44['Adresse_Normalisee'].duplicated(keep=False)]
    
    # Identifier les adresses en doublon dans dfv2
    dfv2_dupes = dfv2[dfv2['Adresse_Normalisee'].duplicated(keep=False)]
    
    # Trouver les adresses en commun entre les deux jeux de données
    common_addresses = set(vf44_dupes['Adresse_Normalisee']).intersection(set(dfv2_dupes['Adresse_Normalisee']))
    
    # Filtrer les données pour ne conserver que celles ayant une adresse en commun
    vf44_common = vf44[vf44['Adresse_Normalisee'].isin(common_addresses)].copy()
    dfv2_common = dfv2[dfv2['Adresse_Normalisee'].isin(common_addresses)].copy()
    
    # Ajouter une colonne Source pour identifier l'origine des données
    vf44_common['Source'] = 'vf44'
    dfv2_common['Source'] = 'dfv2'
    
    # Harmoniser les colonnes des deux dataframes
    all_columns = list(set(dfv2_common.columns).union(set(vf44_common.columns)))
    vf44_common = vf44_common.reindex(columns=all_columns)
    dfv2_common = dfv2_common.reindex(columns=all_columns)
    
    # Concaténer les deux jeux de données sans perte d'information
    all_common_dupes = pd.concat([vf44_common, dfv2_common], ignore_index=True)
    
    # Trier par adresse normalisée pour regrouper les doublons
    all_common_dupes = all_common_dupes.sort_values(by=['Adresse_Normalisee'])
    
    # Exporter en CSV
    all_common_dupes.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Fichier CSV exporté : {output_file}")

export_duplicate_addresses(dfv2, vf44, "adresses_doublon_communes.csv")

/var/folders/bf/ymcp1z4s7n77n5bhz3q_hgqc0000gn/T/ipykernel_3393/2984065862.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_common_dupes = pd.concat([vf44_common, dfv2_common], ignore_index=True)


Fichier CSV exporté : adresses_doublon_communes.csv


On remarque un problème avec les dépendances qui ont les mêmes infos que les appartements dans lesquels elles sont situées. On veut voir quelle quantité elles représentent.

In [30]:
# Filtre
dependance_count = vf44[vf44['Type local'] == 'Dépendance'].shape[0]

print(f"Nombre de lignes avec le type de local 'Dépendance' dans vf44 : {dependance_count}")
# Nombre total de lignes
print(len(vf44['Type local']))

Nombre de lignes avec le type de local 'Dépendance' dans vf44 : 26009
99874
